In [42]:
!pip install -U scikit-learn pandas numpy pillow pyparsing joblib

In [43]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

### Data preparation

In [44]:
df = pd.DataFrame()
for root, _, files in os.walk("data"):
    for filename in files:
        temp_df = pd.read_excel(os.path.join(root, filename), "Sheet1")
        temp_df.drop(columns=[temp_df.columns[0], "Разница в долларах", "Дельта в процентах"], axis=1, inplace=True, errors="ignore")
        df = pd.concat([df, temp_df], axis=0, sort=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24484 entries, 0 to 1497
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Дата           11488 non-null  object 
 1   Время          16445 non-null  object 
 2   Цена до        16445 non-null  float64
 3   Цена после     16445 non-null  float64
 4   Текст новости  16445 non-null  object 
 5   Unnamed: 1     842 non-null    object 
 6   Unnamed: 2     842 non-null    float64
 7   Unnamed: 3     842 non-null    float64
 8   Unnamed: 4     842 non-null    object 
 9   Unnamed: 5     834 non-null    object 
 10  Аннотация      4780 non-null   object 
dtypes: float64(4), object(7)
memory usage: 2.2+ MB


In [45]:
df = df.drop_duplicates(subset=["Текст новости"], keep=False)
df.rename(columns={"Цена до": "price_before", "Цена после": "price_after", "Дата": "date", "Время": "Time",
                   "Текст новости": "news_text", "Аннотация": "annotation"}, inplace=True)

df["absolute_price_difference"] = df["price_after"] - df["price_before"]
df["news_text"] = df["news_text"] + ". " + df["annotation"].fillna("")
df["label"] = np.where(df["absolute_price_difference"] > 0, 1, 0)
df = df[["news_text", "label"]].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6112 entries, 1 to 1009
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   news_text  6112 non-null   object
 1   label      6112 non-null   int32 
dtypes: int32(1), object(1)
memory usage: 119.4+ KB


In [46]:
text_transformer = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))

In [47]:
train_data = df.sample(frac=0.8, random_state=42)

# Testing dataset
test_data = df.drop(train_data.index)

In [48]:
X_train_text = text_transformer.fit_transform(train_data["news_text"])
X_test_text = text_transformer.transform(test_data["news_text"])

In [49]:
X_train_text.shape, X_test_text.shape

((4890, 70088), (52, 70088))

In [50]:
logit_model = LogisticRegression(C=5e1, solver="liblinear", random_state=42)

In [51]:
logit_model.fit(X_train_text, train_data["label"])

LogisticRegression(C=50.0, random_state=42, solver='liblinear')

In [52]:
test_preds = logit_model.predict(X_test_text)

In [53]:
cv_results = cross_val_score(logit_model, X_train_text, train_data["label"], cv=5, scoring="recall_macro")

print(cv_results, cv_results.mean())

[0.58184689 0.60731896 0.59291943 0.56446352 0.60830071] 0.5909699007970611


In [54]:
from sklearn.metrics import accuracy_score, f1_score

print("Logistic Regression accuracy: " + str(accuracy_score(test_data["label"], test_preds)))
print("Logistic Regression f1 score: " + str(f1_score(test_data["label"], test_preds)))

Logistic Regression accuracy: 0.6923076923076923
Logistic Regression f1 score: 0.7241379310344828


In [55]:
# Save our models for deployment

import joblib

Pkl_Filename = "LR_Share_price_prediction_model.pkl"

joblib.dump(logit_model, Pkl_Filename)



['LR_Share_price_prediction_model.pkl']

In [76]:
test_sentence = np.array(["You Ask, We Analyze: Here's The Bull And Bear Case For Advanced Micro Devices Stock"])

test_s = text_transformer.transform(test_sentence)
logit_model.predict_proba(test_s)

array([[0.63848675, 0.36151325]])

In [77]:
test_sentence = np.array(["BofA Hails Nvidia's Dumping Of Arm Takeover"])

test_s = text_transformer.transform(test_sentence)
logit_model.predict_proba(test_s)

array([[0.92791528, 0.07208472]])